In [1]:
import pandas as pd
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize
import pymorphy2
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from gensim.models import LdaModel
from gensim.test.utils import datapath
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt

--2022-04-09 09:19:56--  https://docs.google.com/uc?export=download&confirm=t&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR
Resolving docs.google.com (docs.google.com)... 64.233.162.194, 2a00:1450:4010:c05::c2
Connecting to docs.google.com (docs.google.com)|64.233.162.194|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0hjaktjs921ih2e97cvtcuchc0d6c1q7/1649485125000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download [following]
--2022-04-09 09:19:57--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0hjaktjs921ih2e97cvtcuchc0d6c1q7/1649485125000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 64.233.165.132, 2a00:1450:4010:c08::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleu

In [3]:
!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv

--2022-04-09 09:20:10--  https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a
Resolving drive.google.com (drive.google.com)... 64.233.162.194, 2a00:1450:4010:c05::c2
Connecting to drive.google.com (drive.google.com)|64.233.162.194|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/708a252vqkv6laqj4q2989v1b59cod09/1649485200000/14904333240138417226/*/1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a?e=download [following]
--2022-04-09 09:20:11--  https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/708a252vqkv6laqj4q2989v1b59cod09/1649485200000/14904333240138417226/*/1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a?e=download
Resolving doc-04-c0-docs.googleusercontent.com (doc-04-c0-docs.googleusercontent.com)... 64.233.165.132, 2a00:1450:4010:c08::84
Connecting to doc-04-c0-docs.googleusercontent.com (doc-04-c0-docs.googleuserco

In [4]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2022-04-09 09:20:11--  https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c
Resolving drive.google.com (drive.google.com)... 64.233.162.194, 2a00:1450:4010:c05::c2
Connecting to drive.google.com (drive.google.com)|64.233.162.194|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e3vgkh5sc7uj96sp41i8rda5mesrrec5/1649485200000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download [following]
--2022-04-09 09:20:12--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e3vgkh5sc7uj96sp41i8rda5mesrrec5/1649485200000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 64.233.165.132, 2a00:1450:4010:c08::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleuserco

In [5]:
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

--2022-04-09 09:20:12--  https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK
Resolving drive.google.com (drive.google.com)... 64.233.162.194, 2a00:1450:4010:c05::c2
Connecting to drive.google.com (drive.google.com)|64.233.162.194|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/miah63uarsctseb7t02pnq67ca0ip86o/1649485200000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download [following]
--2022-04-09 09:20:13--  https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/miah63uarsctseb7t02pnq67ca0ip86o/1649485200000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download
Resolving doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs.googleusercontent.com)... 64.233.165.132, 2a00:1450:4010:c08::84
Connecting to doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs.googleuserco

In [6]:
news = pd.read_csv("articles.csv")
users = pd.read_csv("users_articles.csv")

In [7]:
nltk.download('stopwords')
stopword_ru = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/boormistr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopword_ru += additional_stopwords

In [9]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [10]:
tqdm.pandas()
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|          | 0/27000 [00:00<?, ?it/s]<ipython-input-9-898b573b2830>:14: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
100%|██████████| 27000/27000 [00:20<00:00, 1340.90it/s]


In [11]:
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 27000/27000 [02:11<00:00, 205.91it/s] 


In [12]:
texts = list(news['title'].values)
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [13]:
N_topic = 20
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=10)

In [14]:
temp_file = datapath("model.lda")
lda.save(temp_file)
lda = LdaModel.load(temp_file)

In [15]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: сотрудник уголовный орган задержать лицо убийство данные
topic_1: смерть высота родственник умереть горный бежать вниз
topic_2: система земля первый ракета новый статья американский
topic_3: тело взрыв дом квартира памятник французский франция
topic_4: россия российский млрд глава эксперт государство рф
topic_5: китай китайский северный япония японский южный рейтинг
topic_6: проект научный газ новый фонд развитие программа
topic_7: планета пенсия команда девочка игра физика мышь
topic_8: москва первый сайт мероприятие день площадь известный
topic_9: украина военный украинский сила армия территория российский
topic_10: сша россия американский российский санкция европа новый
topic_11: рубль млн рост рынок банк цена тыс
topic_12: долг задолженность бесплатный подсчитать медик ter аналог
topic_13: полоса брюссель парижский приморский чикаго непрерывный прочность
topic_14: снижение белоруссия миссия иран восток сигнал запустить
topic_15: гражданин россиянин россия артист таиланд ро

In [16]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [17]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.000000,0.000000,0.000000,0.000000,0.013482,0.0,0.087853,0.112566,0.010342,...,0.030749,0.031307,0.0,0.000000,0.010443,0.0,0.000000,0.000000,0.0,0.697176
1,4896,0.147673,0.000000,0.000000,0.175095,0.000000,0.0,0.000000,0.434370,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.108574,0.000000,0.0,0.000000
2,4897,0.000000,0.058081,0.000000,0.000000,0.000000,0.0,0.000000,0.604702,0.000000,...,0.000000,0.033123,0.0,0.000000,0.000000,0.0,0.000000,0.283005,0.0,0.000000
3,4898,0.000000,0.000000,0.095446,0.000000,0.000000,0.0,0.000000,0.307694,0.000000,...,0.000000,0.051072,0.0,0.000000,0.012821,0.0,0.130804,0.369308,0.0,0.024501
4,4899,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.179173,0.072669,...,0.222622,0.000000,0.0,0.030978,0.000000,0.0,0.000000,0.000000,0.0,0.472426


In [18]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [19]:
results = pd.DataFrame([['F-Score'], ['Precision'], ['Recall'], ['ROC AUC']], columns=['Metric'])
results = results.set_index('Metric')

In [20]:
target = pd.read_csv("users_churn.csv")

### Max

In [21]:
def get_user_embedding(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(10)

X = pd.merge(user_embeddings, target, 'left')

X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                   X['churn'], random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

preds = logreg.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, preds)
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

results['Max'] = [fscore[ix], precision[ix], recall[ix], auc]

results

,Max
Metric,
F-Score,0.800766
Precision,0.754513
Recall,0.853061
ROC AUC,0.974536


### Median

In [22]:
def get_user_embedding(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(10)

X = pd.merge(user_embeddings, target, 'left')

X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

preds = logreg.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, preds)
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

results['Median'] = [fscore[ix], precision[ix], recall[ix], auc]

results

,Max,Median
Metric,,
F-Score,0.800766,0.804781
Precision,0.754513,0.785992
Recall,0.853061,0.824490
ROC AUC,0.974536,0.974517


### tfidf

In [23]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles'])

TfidfVectorizer()

In [24]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                  'idf': tfidf.idf_})

In [25]:
def get_user_embedding(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight
    
    user_vector = np.median(user_vector, axis=0)
    return user_vector

user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(10)

X = pd.merge(user_embeddings, target, 'left')

X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

preds = logreg.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, preds)
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

results['tfidf'] = [fscore[ix], precision[ix], recall[ix], auc]

results

,Max,Median,tfidf
Metric,,,
F-Score,0.800766,0.804781,0.884696
Precision,0.754513,0.785992,0.909483
Recall,0.853061,0.824490,0.861224
ROC AUC,0.974536,0.974517,0.987690


Лучшим способом оказался tfidf по причине того, что в данном случае выделяются наиболее специализированные новости, которые лучше всего описывают пользователя.